### Youtube video summarizer using wishper and langchain

Workflow:
- Download the YouTube audio file.
- Transcribe the audio using Whisper.
- Summarize the transcribed text using LangChain with three different
 approaches: stuff, refine, and map_reduce.
- Adding multiple URLs to DeepLake database, and retrieving information.

Installation

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = "OPENAI_API"
os.environ['ACTIVELOOP_TOKEN']= "ACTIVELOOP_TOKEN"

In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ewxk8xvb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ewxk8xvb
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=acfadf710491a16784d42338cbc28d7f9d5620d27057567c3f94f772d86ca523
  Stored in directory: /tmp/pip-ephem-wheel-cache-33x8esua/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.8 MB/s eta 0:00:00


In [ ]:
from pytube import YouTube

In [ ]:
link='https://www.youtube.com/watch?v=mBjPyte2ZZo'


In [ ]:
try:
    # object creation using YouTube
    yt = YouTube(link)
except:
    print("Connection Error")

In [ ]:
yt.streams.filter(file_extension='mp4')

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001f" progressive="False" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="133" mime_type="video/mp4" res="240p" fps="30fps" vcodec="avc1.4d4015" progressive="False" type="video">, <Stream: itag="160" mime_type="video/mp4" res="144p" fps="30fps" vcodec="avc1.4d400c" progressive="False" type="video">, <Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">, <Stream: 

In [ ]:
stream = yt.streams.get_by_itag(139)

In [ ]:
stream.download('',"lecuninterview.mp4")

'/content/letmedown.mp4'

In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("lecuninterview.mp4")
print(result['text'])

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 54.8MiB/s]


 Hi, I'm Craig Smith and this is I on A On. This week I talked to Jan LeCoon, one of the seminal figures in deep learning development and a long time proponent of self-supervised learning. Jan spoke about what's missing in large language models and about his new joint embedding predictive architecture which may be a step toward filling that gap. He also talked about his theory of consciousness and the potential for AI systems to someday exhibit the features of consciousness. It's a fascinating conversation that I hope you'll enjoy. Okay, so Jan, it's great to see you again. Good to see you again. I wanted to talk to you about where you've gone with just so supervised learning since last week spoke. In particular, I'm interested in how it relates to large language models because the large language models really came on stream since we spoke. In fact, in your talk about JEPA, which is joint embedding predictive architecture. Thank you. You mentioned that large language models lack a worl

In [ ]:
with open ('text.txt', 'w') as file:
    file.write(result['text'])

In [ ]:
# import yt_dlp

# def download_mp4_from_youtube(url):
#     # Set the options for the download
#     filename = 'lecuninterview.mp4'
#     ydl_opts = {
#         'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
#         'outtmpl': filename,
#         'quiet': True,
#     }

#     # Download the video file
#     with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#         result = ydl.extract_info(url, download=True)

# url = "https://youtu.be/r03JffgFK1Y?si=blrFeUIF9llmq0ZI"
# download_mp4_from_youtube(url)

In [ ]:
from langchain import OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

In [ ]:
from langchain import OpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-instruct",temperature=0)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)

In [ ]:
from langchain.docstore.document import Document

with open('text.txt') as f:
    text = f.read()

texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts[:4]]

In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

llm = ChatOpenAI(temperature=0)

chain = load_summarize_chain(llm, chain_type="map_reduce")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

Jan LeCoon, a prominent figure in deep learning development and advocate for self-supervised
learning, discusses the limitations of large language models and introduces his new joint embedding
predictive architecture. He emphasizes the impact of self-supervised learning on natural language
processing and the importance of transformer architectures. Self-supervised learning involves
training neural networks to predict missing words in text, revolutionizing content moderation
systems. However, generative models struggle to represent uncertain predictions, posing a challenge
when applied to other forms of media like video.


In [ ]:
print( chain.llm_chain.prompt.template )

Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:


In [ ]:
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="stuff",
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary,
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

- Craig Smith interviews Jan LeCoon, a prominent figure in deep learning development and advocate for self-supervised learning.
- Jan discusses the limitations of large language models and introduces his joint embedding predictive architecture (JEPA) as a potential solution.
- Jan also shares his theory of consciousness and the potential for AI systems to exhibit conscious features.
- Self-supervised learning has revolutionized natural language processing by pre-training transformer architectures using text prediction.
- Large language models can generate text spontaneously, but struggle with representing uncertain predictions.
- Jan's JEPA aims to address this limitation and expand the application of self-supervised learning to other domains like video analysis.


In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

Jan LeCoon, a prominent figure in deep learning development and advocate for self-supervised
learning, discusses the significance of self-supervised learning in revolutionizing natural language
processing and the use of transformer architectures. He explains the process of training large
neural nets to predict missing words in text, which results in the system learning good
representations of text that can be used for various downstream tasks. Jan also introduces his new
joint embedding predictive architecture, which addresses the limitations of large language models.
Additionally, he shares that self-supervised learning has been widely adopted in practical
applications such as contact moderation systems on platforms like Facebook, Google, and YouTube. The
limitations of generative models in representing uncertain predictions are highlighted, particularly
in the case of applying self-supervised learning to video analysis.


In [ ]:
# import yt_dlp

# def download_mp4_from_youtube(urls, job_id):
#     # This will hold the titles and authors of each downloaded video
#     video_info = []

#     for i, url in enumerate(urls):
#         # Set the options for the download
#         file_temp = f'./{job_id}_{i}.mp4'
#         ydl_opts = {
#             'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
#             'outtmpl': file_temp,
#             'quiet': True,
#         }

#         # Download the video file
#         with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#             result = ydl.extract_info(url, download=True)
#             title = result.get('title', "")
#             author = result.get('uploader', "")

#         # Add the title and author to our list
#         video_info.append((file_temp, title, author))

#     return video_info

# urls=["https://www.youtube.com/watch?v=mBjPyte2ZZo&t=78s",
#     "https://www.youtube.com/watch?v=cjs7QKJNVYM",]
# vides_details = download_mp4_from_youtube(urls, 1)

In [ ]:
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "vishalug3016"
my_activeloop_dataset_name = "langchain_course_youtube_summarizer"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


Creating 4 embeddings in 1 batches of size 4:: 100%|██████████| 1/1 [00:06<00:00,  6.05s/it]

Dataset(path='hub://vishalug3016/langchain_course_youtube_summarizer', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   


['63ebfa9a-c2e5-11ee-ba24-0242ac1c000c',
 '63ebfc3e-c2e5-11ee-ba24-0242ac1c000c',
 '63ebfd10-c2e5-11ee-ba24-0242ac1c000c',
 '63ebfdce-c2e5-11ee-ba24-0242ac1c000c']

In [ ]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['k'] = 4

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of transcripts from a video to answer the question in bullet points and summarized. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Summarized answer in bullter points:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 chain_type_kwargs=chain_type_kwargs)

print( qa.run("Summarize the mentions of google according to their AI program") )

- Google is mentioned as one of the companies that uses self-supervised learning in their top-performing contact moderation systems.
- The speaker mentions that large language models, including those used by Google, lack the ability to represent uncertain predictions.
- There is no specific mention of Google's AI program in relation to self-supervised learning.
